<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/EncodeDecoder_Test_0825.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 여기서부터 TEST

참조 : https://github.com/kiyoungkim1/LMkor


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
#from torchtext.datasets import Multi30k
#from torchtext.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
import urllib.request
import nltk
nltk.download('punkt')

# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    #txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    #txt = txt.replace('.','. ')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def collect_training_dataset_for_s2s_generator(source_urls=[]):
    ko_sentences_dataset = []
    for url in source_urls:
        raw_text = urllib.request.urlopen(url).read().decode('utf-8')
        ko_sentences_dataset += nltk.sent_tokenize(clean_text(raw_text))

    sentences = []
    for txt in ko_sentences_dataset:
        txt = txt.strip()
        if len(txt) > 40 and txt.endswith('다.'):
            #ko_grammar_dataset.append([txt,1])
            txt = txt.replace('.','')
            sentences.append(txt)

    return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv')
df

,title,contents
0,문 대통령 변창흠 국토장관 사의표명 사실상 수용,정만호 국민소통수석이 12일 오후 청와대 춘추관 대브리핑룸에서 변창흠 국토부 장관 ...
1,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
2,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
3,수상자 메달 걸어주는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
4,정몽구 서울아산병원에 50억 쾌척,인재 양성·소외 계층 지원 등 계획 “부친 질병·가난 악순환 끊기 원해 국내 최고 ...
...,...,...
140559,[건축과도시] 북한산을 캔버스 삼아. 미술관 또 하나의 작품이 되다,<은평구 진관동 사비나 미술관> 서울시 은평구 진관동에 자리잡은 사비나미술관. 삼각...
140560,조선후기 문인 김조순 별장 그린 옥호정도 첫 공개,국립중앙박물관 서화실 개편해 32점 새롭게 전시 옥호정도[국립중앙박물관 제공연합뉴스...
140561,안성 청룡사 대웅전에서 목재 곡자 발견,문화재청(청장 정재숙)의 국고보조와 기술지도로 안성시에서 시행하고 있는 안성 청룡사...
140562,156년전 ㄱ자 곡자 찾았다 안성 청룡사 기둥 밑에서,안성 청룡사 대웅전에서 발견된 곡자 【서울뉴시스】 이수지 기자 안성 청룡사 대웅전에...


In [7]:
import re
import sys
# 검사...
pattens = ["[34569][0-9]{3}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}",
           "[0-9]{2,3}[\:\s\;.\;,\;-;)][0-9]{3,4}[\:\s\;.\;,\;-][0-9]{4}",
           "[0-9]{1}[0-9]{1}[\W]?[0-1]{1}[0-9]{1}[\W]?[0-3]{1}[\W]?[0-9]{1}[\W]?[1-4]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}",
           "[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{3}[\:\s\;.\;,\;-]([0-9]{5,6}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{5}|[0-9]{2,3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{7}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{4,6}[\:\s\;.\;,\;-][0-9]|[0-9]{5}[\:\s\;.\;,\;-][0-9]{3}[\:\s\;.\;,\;-][0-9]{2}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{5}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{4}[\:\s\;.\;,\;-][0-9]{4}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{7})|[0-9]{4}[\:\s\;.\;,\;-]([0-9]{3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{6}[\:\s\;.\;,\;-][0-9])|[0-9]{5}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{5,6}"
           ]

filters = []
for p in pattens:
    filters.append(re.compile(p))

sentences = []

for raw_text in df['contents']:
    try:
        docs = nltk.sent_tokenize(clean_text(raw_text))
        for txt in docs:
            if txt.find('▶') > -1 or txt.find('@') > -1 or txt.find('ⓒ') > -1: 
                pass
            else:
                txt = txt.strip()
                if len(txt) > 40:
                    #ok = True
                    #for f in filters:
                    #    if len(f.findall(txt)) > 0:
                    #        ok = False
                    #        print('fitered:',txt)
                    #if ok:
                    if any(chr.isdigit() for chr in txt):
                        pass
                    else:
                        sentences.append(txt)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        print("Unexpected error:", sys.exc_info()[0])      



Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected 

In [8]:
len(sentences)

1544643

In [9]:
import re
import sys
import io

#텍스트 정제(전처리)
def cleanText(readData):
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', readData)
    return text

In [10]:
c_sentences = []
for sentence in sentences:
    s = cleanText(sentence)
    if len(s.split()) < 10:
        c_sentences.append(s)

In [11]:
len(c_sentences)

58552

In [ ]:
c_sentences[1]

In [12]:
!pip3 install keybert
!pip3 install sentence-transformers
!pip3 install -q transformers

In [13]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("xlm-r-large-en-ko-nli-ststb")

In [14]:
key_model = KeyBERT(model=sentence_model)

In [15]:
def extract_key(sentence):
    #print(s)
    top_n= int(len(sentence.split())/3)
    keys = key_model.extract_keywords(sentence,top_n=top_n)
    x= {}
    for k,p in keys:
        x[k] = sentence.lower().find(k)
        #print(x[k],k)
    keys = list(dict(sorted(x.items(), key=lambda item: item[1])).keys())
    return keys




In [16]:
i = 200
[' '.join(extract_key(c_sentences[i])),c_sentences[i]]

['가입자 사건에 한해서라도', '가입자 단체에서는 일부 사건에 한해서라도 제도 도입이 필요하다고 주장합니다']

In [17]:
pretraoned_kobert_model_name='kykim/bert-kor-base'
#tokenizer = KoBertTokenizer.from_pretrained(pretraoned_kobert_model_name)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
from transformers import EncoderDecoderModel, BertTokenizerFast
import torch

tokenizer = BertTokenizerFast.from_pretrained(pretraoned_kobert_model_name)
model = EncoderDecoderModel.from_encoder_decoder_pretrained(pretraoned_kobert_model_name, pretraoned_kobert_model_name) # initialize Bert2Bert from pre-trained checkpoints


Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertLMHeadModel: ['cls.seq_re

In [20]:
op = tokenizer("김성수 과학기술정보통신부 과학기술혁신본부장이 간담회 참석자들과 기념촬영 하고 있다", return_tensors="pt")
print(op)
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in op['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(op['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(op['attention_mask'].tolist()[0]))

{'input_ids': tensor([[    2, 24499,  8069, 35077, 15954, 18931,  8210, 35077, 31246, 22116,
         14248, 31650,  8113, 17287, 30246, 16022, 19550, 14042, 13984,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Tokens (str)      : ['[CLS]', '김성', '##수', '과학기술', '##정보', '##통신', '##부', '과학기술', '##혁신', '##본부', '##장이', '간담', '##회', '참석', '##자들과', '기념', '##촬영', '하고', '있다', '[SEP]']
Tokens (int)      : [2, 24499, 8069, 35077, 15954, 18931, 8210, 35077, 31246, 22116, 14248, 31650, 8113, 17287, 30246, 16022, 19550, 14042, 13984, 3]
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



In [21]:
class Batch:
  src = []
  trg = []
max_length = 100
batch_size = 8 #4 # 64, 128
dataset_iterator = []
batch_counter = 0

sentences2 = c_sentences[:1000]
for i in range(0,len(sentences2),batch_size):
#for i in range(0,1,batch_size):
    nSRC = []
    nTRG = []
    src_max = 0
    trg_max = 0
    batch = Batch()
    batch.src = []
    batch.trg = []
    
    percent = ("{0:.2f}").format(100 * (i / float(len(sentences2))))
    print(f'\r {percent}% {i}/{str(len(sentences2))}', end="", flush=True)
    for j in range(i,i+batch_size):
        try:
            sentence = sentences2[j]+'.'
            keys = extract_key(sentence)
            srct = tokenizer((' '.join(keys)).strip(), return_tensors="pt")['input_ids'].tolist()[0]
            trgt = tokenizer(sentence, return_tensors="pt")['input_ids'].tolist()[0]
            
            #srct = tokenizer.convert_tokens_to_ids(['[CLS]'] + tokenizer.tokenize((' '.join(keys)).strip()) + ['[SEP]'])
            #trgt = tokenizer.convert_tokens_to_ids(['[CLS]'] + tokenizer.tokenize(sentence ) + ['[SEP]'])
            if len(srct) < max_length and len(trgt) < max_length:
                if len(trgt) > trg_max:
                    trg_max = len(trgt)            
                nTRG.append(trgt)
                if len(srct) > src_max:
                    src_max = len(srct)
                nSRC.append(srct)

        except Exception as e:
            print('Error:',e)
            pass

    for s in nSRC:
        ss = s
        if len(s) < src_max:
            ss += [1 for i in range(src_max-len(s))]
        batch.src.append(ss)

    for t in nTRG:
        tt = t
        if len(t) < trg_max:        
            tt += [1 for i in range(trg_max-len(t))]
        batch.trg.append(tt)

    #print(len(batch.src),len(batch.src[0]))
    #print(len(batch.trg),len(batch.trg[0]))

    batch.src = torch.tensor(batch.src).to(device)
    batch.trg = torch.tensor(batch.trg).to(device)
    dataset_iterator.append(batch)



 99.20% 992/1000

In [23]:
dataset_iterator[0].src

tensor([[    2, 29110,  8269, 18189, 22327,  8061, 25810, 16301,     3,     1,
             1,     1,     1],
        [    2, 35077, 15954, 18931,  8210, 35077, 31246, 22116, 14248,     3,
             1,     1,     1],
        [    2, 39009, 21108, 17608,  2784,  8069, 15880, 16207,     3,     1,
             1,     1,     1],
        [    2, 17497,  8323,  8040, 13973, 40282, 14062,     3,     1,     1,
             1,     1,     1],
        [    2,  5921,  8146, 25273, 15093,  8159,  5921, 17400, 19913,  8048,
          4047, 15804,     3],
        [    2,  5921, 17400, 19913, 19974, 14350, 37562,     3,     1,     1,
             1,     1,     1],
        [    2, 27757,  8273, 16379, 37900,  8048, 16683, 40986,  8048,     3,
             1,     1,     1],
        [    2, 31179, 14004, 15436, 15534, 17543, 14399,     3,     1,     1,
             1,     1,     1]], device='cuda:0')

# Train 준비

In [24]:
SRC_PAD_IDX = tokenizer.pad_token_id
TRG_PAD_IDX = tokenizer.pad_token_id

LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [33]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        ##print(src.shape)
        ##print(trg.shape)
        outputs = model(input_ids=src, decoder_input_ids=trg, labels=trg)
        #print(outputs)
        loss, logits = outputs.loss, outputs.logits

        ##print(output.shape)
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        #output_dim = output.shape[-1]
            
        #output = output.contiguous().view(-1, output_dim)
        #trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        #loss = criterion(output, trg)
        
        loss.backward()
        
        ##print('loss',loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        ##print('epoch_loss',epoch_loss)
    return epoch_loss / len(iterator)

In [26]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Train !!!!

In [28]:
bcnt = len(dataset_iterator)
print(bcnt)
train_iterator = dataset_iterator[0:bcnt-int(bcnt/10)]
valid_iterator = dataset_iterator[bcnt-int(bcnt/10):bcnt]
#train_iterator = dataset_iterator[0:bcnt-1]
#valid_iterator = dataset_iterator[bcnt-1:bcnt]
print('train',len(train_iterator))
print('valid',len(valid_iterator))

125
train 113
valid 12


In [29]:
train_iterator[0].src

tensor([[    2, 29110,  8269, 18189, 22327,  8061, 25810, 16301,     3,     1,
             1,     1,     1],
        [    2, 35077, 15954, 18931,  8210, 35077, 31246, 22116, 14248,     3,
             1,     1,     1],
        [    2, 39009, 21108, 17608,  2784,  8069, 15880, 16207,     3,     1,
             1,     1,     1],
        [    2, 17497,  8323,  8040, 13973, 40282, 14062,     3,     1,     1,
             1,     1,     1],
        [    2,  5921,  8146, 25273, 15093,  8159,  5921, 17400, 19913,  8048,
          4047, 15804,     3],
        [    2,  5921, 17400, 19913, 19974, 14350, 37562,     3,     1,     1,
             1,     1,     1],
        [    2, 27757,  8273, 16379, 37900,  8048, 16683, 40986,  8048,     3,
             1,     1,     1],
        [    2, 31179, 14004, 15436, 15534, 17543, 14399,     3,     1,     1,
             1,     1,     1]], device='cuda:0')

In [34]:

N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')
model.to(device)

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    #valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    #if valid_loss < best_valid_loss:
    #    best_valid_loss = valid_loss
    #    torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 16s
	Train Loss: 6.398 | Train PPL: 600.532
Epoch: 02 | Time: 0m 16s
	Train Loss: 6.172 | Train PPL: 479.333
Epoch: 03 | Time: 0m 16s
	Train Loss: 6.069 | Train PPL: 432.205


KeyboardInterrupt: ignored

In [ ]:
def complete_sentence(sentence, model, device, max_len = 50):
    
    model.eval()
    '''
    if isinstance(sentence, str):
        nlp = spacy.load('de_core_news_sm')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    '''
    #tokens = tokenizer.tokenize('[CLS] ' + sentence + ' [SEP]')
    #tokens = [tokenizer.bos_token] + tokens + [tokenizer.eos_token]
    

    #src_indexes = [tokenizer.convert_tokens_to_ids(token) for token in tokens]
    src_indexes = tokenizer(sentence, return_tensors="pt")['input_ids'].tolist()[0]
    print(src_indexes)
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    print(src_tensor)
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [tokenizer.cls_token_id]
    

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == tokenizer.sep_token_id: #tk.stoi['<eos>']: #.eos_token_id: # trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = tokenizer.convert_ids_to_tokens(trg_indexes) # :[tk.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [ ]:
trg, _ = complete_sentence('북한 미국 미사일',model,device)

In [ ]:
print(''.join(trg))

In [ ]:
extract_key(sentences[101])

In [ ]:
trg, _ = complete_sentence('가입자 사건에 한해서라도',model,device)
print(' '.join(trg))

In [42]:
keys = '가입자 사건에 한해서라도'
input_ids = torch.tensor(tokenizer.encode(keys, add_special_tokens=True)).unsqueeze(0)
#input_ids.to(device)
print(input_ids)
model.eval()
model.cpu()
generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.pad_token_id)
generated

tensor([[    2, 16704,  8158, 31829, 26980,  8044, 14566,     3]])


tensor([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])